# Function Tests

This notebook tests `countmatch_common.py`, `countmatch_teps.py` and `countmatch_bagheri.py`.

In [1]:
%matplotlib inline
import sys
sys.path.append('../')
import importlib
import matplotlib.pyplot as plt
import numpy as np
import knowyourdata as kyd

import pandas as pd
from traffic_prophet import cfg
import pathlib, os
import configparser

from traffic_prophet import connection
from traffic_prophet.countmatch import reader
from traffic_prophet.countmatch import growthfactor as gf
from traffic_prophet.countmatch import neighbour

defaultcolours = plt.rcParams['axes.prop_cycle'].by_key()['color']

filepath = pathlib.Path.home().joinpath('.charlesconfig')
if os.path.isfile(filepath):
    vol_conn = connection.Connection(filepath, 'POSTGRES',
                                     'czhu.btp_centreline_daily_counts')
    ll_conn = connection.Connection(filepath, 'POSTGRES',
                                    'czhu.btp_centreline_lonlat')
    config = configparser.RawConfigParser()
    config.read(filepath.as_posix())
    MAPBOX_TOKEN = config['MAPBOX']['token']
    PLOTLY_USER = config['PLOTLY']['user']
    PLOTLY_KEY = config['PLOTLY']['key']
else:
    filepath = pathlib.Path.home().joinpath('cf.txt')
    vol_conn = connection.Connection(filepath, 'localpg',
                                     'prj_vol.btp_centreline_daily_counts')
    ll_conn = connection.Connection(filepath, 'localpg',
                                    'gis.btp_centreline_lonlat')
    config = configparser.RawConfigParser()
    config.read(filepath.as_posix())
    MAPBOX_TOKEN = config['mapbox']['token']
    PLOTLY_USER = config['plotly']['user']
    PLOTLY_KEY = config['plotly']['apikey']

In [2]:
%%time
rdr_teps = reader.Reader(vol_conn)
rdr_teps.read()
gf.get_growth_factors(rdr_teps)

CPU times: user 1min 55s, sys: 3 s, total: 1min 58s
Wall time: 2min 8s


In [3]:
ptc_ids = np.unique(np.abs(list(rdr_teps.ptcs.keys())))
nb = neighbour.NeighbourLonLatEuclidean(ll_conn, 20, ptc_ids)
%time nb.find_neighbours()

CPU times: user 18.6 s, sys: 156 ms, total: 18.8 s
Wall time: 18.7 s


In [4]:
%%time
rdr_bag = reader.Reader(vol_conn)
rdr_bag.read()
gf.get_growth_factors(rdr_bag)

CPU times: user 2min 1s, sys: 3.23 s, total: 2min 4s
Wall time: 2min 14s


## TEPs-like CountMatch

Testing the `countmatch_teps` module.

In [ ]:
import countmatch_teps as cmt
importlib.reload(cmt)

In [ ]:
teps_aadt_estimates = cmt.estimate_aadts(
    rdr_teps, nb, 2019, n_neighbours=5, single_direction=True, progress_bar=True)

In [8]:
import countmatch_common as cmc
importlib.reload(cmc)

<module 'countmatch_common' from '/mnt/c/Users/czhu5/Documents/Ubuntu/GitHub/bdit_traffic_prophet/development_notebooks/countmatch_common.py'>

In [9]:
ptc_estimates = cmc.aadt_estimator_for_ptcs(rdr_teps, 2019, override_growth_factor=False)

In [ ]:
teps_aadt_estimates = cmc.combine_estimators(teps_aadt_estimates, ptc_estimates)

In [ ]:
teps_aadt_estimates

## Hybrid Count Module

Testing `countmatch_hybrid.py`.

In [5]:
import countmatch_hybrid as cmh
importlib.reload(cmh)

<module 'countmatch_hybrid' from '/mnt/c/Users/czhu5/Documents/Ubuntu/GitHub/bdit_traffic_prophet/development_notebooks/countmatch_hybrid.py'>

In [6]:
bag_aadt_estimates = cmh.estimate_aadts(rdr_bag, nb, 2019, progress_bar=True)

In [10]:
bag_aadt_estimates.head()

,Count ID,PTC ID,Growth Factor,Most Recent Year,AADT Estimate
0,-175,-1022,1.135230,2010,5378.846957
1,175,444516,1.271847,2010,12889.208626
2,-201,-445884,0.909007,2006,74.947454
3,201,444782,0.984669,2006,157.838043
4,-202,-445884,0.909007,2006,49.870941


In [11]:
bag_aadt_estimates = cmc.combine_estimators(bag_aadt_estimates, ptc_estimates)

In [12]:
bag_aadt_estimates

,Count ID,PTC ID,Growth Factor,Most Recent Year,AADT Estimate_STTC,AADT Estimate_PTC,AADT Estimate
0,-175,-1022,1.135230,2010,5378.846957,NaN,5378.846957
1,175,444516,1.271847,2010,12889.208626,NaN,12889.208626
2,-201,-445884,0.909007,2006,74.947454,NaN,74.947454
3,201,444782,0.984669,2006,157.838043,NaN,157.838043
4,-202,-445884,0.909007,2006,49.870941,NaN,49.870941
5,202,444782,0.984669,2006,146.990007,NaN,146.990007
6,-214,-890,1.038726,2011,3809.125278,NaN,3809.125278
7,-229,-445884,0.909007,2006,78.842754,NaN,78.842754
8,229,444782,0.984669,2006,180.716896,NaN,180.716896
9,-256,-445884,0.909007,2006,3637.407741,NaN,3637.407741


## Bagheri Count Module